In [1]:
# 1. 필요한 라이브러리 불러오기 및 종목코드(stock_code)와 재무제표 종류(rpt_type) 설정
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request

In [16]:
# 2. 손익계산서 각 항목별 최근 3개 연간 데이터를 저장할 수 있는 리스트 설정
# [FnGuide] 공시기업의 최근 3개 연간 및 4개 분기 손익계산서를 수집하는 함수
def getBS(stock_code, rpt_type, freq) :
    items_en = ['assets', 'curassets', 'curassets1', 'curassets2', 'curassets3',  'curassets4', 'curassets5',
               'curassets6', 'curassets7', 'curassets8', 'curassets9', 'curassets10', 'curassets11',
               'ltassets', 'ltassets1', 'ltassets2', 'ltassets3', 'ltassets4', 'ltassets5', 'ltassets6', 'ltassets7', 
                'ltassets8', 'ltassets9', 'ltassets10', 'ltassets11', 'ltassets12', 'ltassets13', 'finassets',
                'liab', 'curliab', 'curliab1', 'curliab2', 'curliab3', 'curliab4', 'curliab5', 
                'curliab6', 'curliab7', 'curliab8', 'curliab9', 'curliab10', 'curliab11', 'curliab12', 'curliab13', 
                'ltliab', 'ltliab1', 'ltliab2', 'ltliab3', 'ltliab4', 'ltliab5', 'ltliab6', 
                'ltliab7', 'ltliab8', 'ltliab9', 'ltliab10', 'ltliab11', 'ltliab12', 'finliab',
                'equity', 'equity1', 'equity2', 'equity3', 'equity4', 'equity5', 'equity6', 'equity7', 'equity8']

# 3. 연결 재무상태표 URL 및 항목 설정
    if rpt_type.upper() == 'CONSOLIDATED' : # 연결 연간 손익계산서(ReportGB=D)
        url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701".format(stock_code)
    else : # 별도 연간 손익계산서(ReportGB=B)
        url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701".format(stock_code)
        items_en = [item for item in items_en if item not in ['equity1', 'equity8']]
        
# 4. 데이터 요청 및 결과 확인
    # Header 설정
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36',
    }
    req = Request(url=url, headers=headers)
    html = urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    if freq.upper() == 'A' : # 연간 재무상태표 영역 추출
        is_a = soup.find(id = 'divDaechaY')
        num_col = 3 # 최근 3개 연간 데이터
    else : # 분기 재무상태표 영역 추출 freq.upper() == 'Q'
        is_a = soup.find(id = 'divDaechaQ')
        num_col = 4 # 최근 4개 분기 데이터
    bs_a = is_a.find_all(['tr'])
    # print(bs_a)
    
# 5. 접힌 항목을 모두 펼친 뒤에 하위 항목 추출
    # 연간 손익계산서 항목 펼친 뒤 하위 항목 추출
    items_kr = [bs_a[m].find(['th']).get_text().replace('\n','').replace('\xa0','').replace('계산에 참여한 계정 펼치기','')
                for m in range(1, len(bs_a))]
    # print(items_kr)
    
# 6. 최근 3개 연도 손익계산서 발행년월(period) 수집
    # 최근 3개 연간 손익계산서 자료 수집
    period = [bs_a[0].find_all('th')[n].get_text() for n in range(1, num_col+1)]
    # print(period)
    
# 7. 맨 처음 설정했던 변수들에 각각 저장
# 항목별 최근 3개 연간데이터 불러오기
    for item, i in zip(items_en, range(1, len(bs_a))):
    
        temps = []
        for j in range(0, num_col):
            temp = [float(bs_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0',''))\
                   if bs_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') != ''\
                    else (0 if bs_a[i].find_all('td')[j]['title'].replace(',','').replace('\xa0','') == '-0'\
                         else 0)]
            temps.append(temp[0])
        # item_en 내 각 항목을 global 변수로 지정하고 값 저장
        globals()[item] = temps
    # print(assets) # 총 자산
    
 # 8. 연결 및 별도에 따라 지배/비지배주주순이익 변수 처리
    # 지배/비지배 항목 처리
    if rpt_type.upper() == 'CONSOLIDATED' : # 연결 연간 재무상태표는 아무 것도 하지 않음
        pass
    else : # 별도 연간 재무상타표는 연간에만 존재하는 항목을 Null 값으로 채움
        globals()['equity1'], globals()['equity8'] = [np.NaN]*num_col, [np.NaN]*num_col
    # print(equity1, equity2)
    
# 9. 재무상태표 테이블 컬럼과 앞에서 정의한 global 변수를 각각 매칭하여 DataFrame으로 변환
    bs_domestic = pd.DataFrame({'stock_code':stock_code, 'period':period,
                            'Assets_Total' : assets, "Current_Assets_Total" : curassets,
                            'LT_Assets_Total' : Itassets, "Other_Fin_Assets" : finassets,
                            'Liabilities_Total' : liab, "Current_Liab_Total" : curliab,
                            'LT_Liab_Total' : ltliab, "Other_Fin_Liab_Total" : finliab,
                            "Equity_Total" : equity, "Controlling_Equity_Total" : equity1,
                            "Non_Controlling_Equity_Total" : equity8})
    # 재무제표 종류 컬럼 추가
    bs_domestic['rpt_type'] = rpt_type + '_' + freq.upper()
# 10. 재무상태표 테이블 컬럼과 앞에서 정의한 global 변수를 각각 매칭하여 Dataframe으로 변환
    return bs_domestic

In [17]:
getBS('005930', 'Consolidated', 'A') # 삼성전자(005930)의 연결, 연간 (포괄) 재무상태표

,stock_code,period,Assets_Total,Current_Assets_Total,LT_Assets_Total,Other_Fin_Assets,Liabilities_Total,Current_Liab_Total,LT_Liab_Total,Other_Fin_Liab_Total,Equity_Total,Controlling_Equity_Total,Non_Controlling_Equity_Total,rpt_type
0,005930,2018/12,3393572.44,1746974.24,1646598.20,0,916040.67,690815.10,225225.57,0,2477531.77,2400689.93,76841.84,Consolidated_A
1,005930,2019/12,3525644.97,1813852.60,1711792.37,0,896840.76,637827.64,259013.12,0,2628804.21,2549154.72,79649.49,Consolidated_A
2,005930,2020/12,3782357.18,1982155.79,1800201.39,0,1022877.02,756043.51,266833.51,0,2759480.16,2676703.31,82776.85,Consolidated_A
